#Description
This notebook aims to get the NPL information from BACEN timeseries system

# Setup

In [1]:
import pandas as pd
import requests
import plotly.express as px
import io
import datetime

# Data

In [2]:
requests_dict = {
    'npl_individuals_total' : '21033',
    'npl_overdraft' : '21034',
    'npl_personal_loans' : '21035',
    'npl_private_payroll' : '21037',
    'npl_public_sector_payroll' : '21038',
    'npl_retirees_payroll' : '21039',
    'npl_payroll_total' : '21041',
    'npl_credit_cards' : '21050',
}

def getBacenSeries(dict_series: dict, end_date: str, beg_date: str = '01/01/1995'):
    result = None
    for name_i, code_i in dict_series.items():
        url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{code_i}/dados?formato=csv&dataInicial={beg_date}&dataFinal={end_date}"
        df = pd.read_csv(url, sep=";", decimal=",", date_format="%d/%m/%Y", parse_dates=[0])
        df = df.rename({'valor':name_i, 'data' : 'date'}, axis='columns')
        if result is None:
            result = df
        else:
            result = result.merge(df, on='date', how='left')
    return result
        
npls_df = getBacenSeries(requests_dict, "31/12/2025")
npls_df.head()

,date,npl_individuals_total,npl_overdraft,npl_personal_loans,npl_private_payroll,npl_public_sector_payroll,npl_retirees_payroll,npl_payroll_total,npl_credit_cards
0,2011-03-01,6.83,5.49,5.67,5.75,5.21,1.79,4.70,4.17
1,2011-04-01,6.78,5.67,5.39,6.38,4.71,1.75,4.43,4.16
2,2011-05-01,6.62,5.19,5.34,4.86,4.55,1.83,4.28,4.19
3,2011-06-01,6.61,5.76,5.28,4.81,4.43,1.74,4.20,4.28
4,2011-07-01,6.53,6.41,5.02,4.75,4.06,1.85,3.97,4.18


In [3]:
data_chart = npls_df.melt(id_vars='date')
fig = px.line(data_chart,
              x='date',
              y='value',
              color='variable',
              template='none',
              width=1000)
fig